In [1]:
!mkdir -p ~/.kaggle

In [2]:
!cp kaggle.json ~/.kaggle

In [3]:
!kaggle datasets download -d hemooredaoo/cataract

 99% 547M/552M [00:05<00:00, 146MB/s]
100% 552M/552M [00:05<00:00, 109MB/s]


In [4]:
!unzip '/content/cataract.zip'

Archive:  /content/cataract.zip
  inflating: cataract-image-dataset/processed_images/test/cataract/image_246.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_247.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_248.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_249.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_250.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_251.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_252.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_253.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_254.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_255.png  
  inflating: cataract-image-dataset/processed_images/test/cataract/image_256.png  
  inflating: cataract-image-dataset/processed_images/te

In [86]:


train_directory = '/content/cataract-image-dataset/processed_images/train'
test_directory = '/content/cataract-image-dataset/processed_images/test'




In [87]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [114]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,rotation_range=40,
    fill_mode='nearest',horizontal_flip=True,shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [115]:
training_set = train_datagen.flow_from_directory(
    directory = train_directory,
    target_size = (224, 224),
    batch_size = 32,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    directory = test_directory,
    target_size = (224, 224),
    batch_size = 32,
    class_mode='binary'
)

Found 978 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


In [116]:
len(training_set), len(test_set)

(31, 4)

In [117]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.models import Model


In [118]:
vgg = VGG16(weights="imagenet",include_top=False,input_shape=(224,224,3))

In [119]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [120]:
for layer in vgg.layers:
  print(layer)

In [121]:
len(vgg.layers)

19

In [122]:
for layer in vgg.layers:
    layer.trainable = False

In [123]:
x = Flatten()(vgg.output)

In [126]:
output = Dense(1,activation='sigmoid')(x)

In [127]:
vgg16 = Model(vgg.input,output)

In [128]:
vgg16.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [153]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'best_vgg16.h5',  # file name to save the best model
    monitor='val_accuracy',  # metric to monitor
    save_best_only=True,  # save only the best model
    mode='max',  # mode can be 'max' for accuracy, 'min' for loss, etc.
    verbose=1  # optional, set to 1 to see progress during training
)


In [130]:
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
opt = Adam(learning_rate=0.001)

# Assuming you have defined your VGG16 model as vgg16

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Compile the model (you may have already done this)
vgg16.compile(optimizer='Adam' , loss='binary_crossentropy', metrics=['accuracy'])

#Train the model with early stopping callback
vgg16.fit(training_set, validation_data=test_set, epochs=50, steps_per_epoch=len(training_set), validation_steps=len(test_set), callbacks=[early_stopping,checkpoint])

Epoch 1/50
31/31 [==============================] - ETA: 0s - loss: 0.5094 - accuracy: 0.7597
Epoch 1: val_accuracy improved from -inf to 0.88430, saving model to best_vgg16.h5
31/31 [==============================] - 34s 1s/step - loss: 0.5094 - accuracy: 0.7597 - val_loss: 0.2259 - val_accuracy: 0.8843
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - ETA: 0s - loss: 0.3004 - accuracy: 0.8753
Epoch 2: val_accuracy improved from 0.88430 to 0.93388, saving model to best_vgg16.h5
31/31 [==============================] - 30s 993ms/step - loss: 0.3004 - accuracy: 0.8753 - val_loss: 0.1790 - val_accuracy: 0.9339
Epoch 3/50
31/31 [==============================] - ETA: 0s - loss: 0.2468 - accuracy: 0.9070
Epoch 3: val_accuracy improved from 0.93388 to 0.94215, saving model to best_vgg16.h5
31/31 [==============================] - 30s 960ms/step - loss: 0.2468 - accuracy: 0.9070 - val_loss: 0.1571 - val_accuracy: 0.9421
Epoch 4/50
31/31 [==============================] - ETA: 0s - loss: 0.2243 - accuracy: 0.9131
Epoch 4: val_accuracy improved from 0.94215 to 0.95041, saving model to best_vgg16.h5
31/31 [==============================] - 30s 976ms/step - loss: 0.2243 - accuracy: 0.9131 - val_loss: 0.1245 - val_accuracy: 0.9504
Epoch 5/50
31/31 [==============================] - ETA: 0s - loss: 0.1984 

In [155]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [168]:
img_path ='/content/cataract-image-dataset/processed_images/test/cataract/image_289.png'

In [169]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds
if preds< 0.5:
    print('Cataract')
else:
    print('Normal')

1/1 [==============================] - 0s 24ms/step
Cataract


In [163]:
img_path='/content/cataract-image-dataset/processed_images/test/normal/image_250.png'

In [165]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds
if preds< 0.5:
    print('Cataract')
else:
    print('Normal')

1/1 [==============================] - 0s 18ms/step
Normal


In [167]:
vgg16.save('Vgg16_97.h5')

In [170]:
# VGG19
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [171]:
vgg19 = VGG19(include_top=False ,input_shape = (224,224,3))

In [173]:
for layer in vgg19.layers:
  print(layer)

In [174]:
len(vgg19.layers)

22

In [175]:
for layer in vgg19.layers:
  layer.trainable = False

In [176]:
x = Flatten()(vgg19.output)

In [177]:
output = Dense(1,activation = 'sigmoid')(x)

In [178]:
vgg19 =Model(vgg19.input,output)

In [152]:
vgg19.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [179]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'best_vgg19.h5',  # file name to save the best model
    monitor='val_accuracy',  # metric to monitor
    save_best_only=True,  # save only the best model
    mode='max',  # mode can be 'max' for accuracy, 'min' for loss, etc.
    verbose=1  # optional, set to 1 to see progress during training
)
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
opt = Adam(lr=0.0001)

# Assuming you have defined your VGG16 model as vgg16

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Compile the model (you may have already done this)
vgg19.compile(loss = 'binary_crossentropy',optimizer = opt,metrics =['accuracy'])

# Train the model with early stopping callback
history = vgg19.fit(training_set, validation_data=test_set, epochs=50, steps_per_epoch=len(training_set), validation_steps=len(test_set), callbacks=[checkpoint,early_stopping])

Epoch 1/50
31/31 [==============================] - ETA: 0s - loss: 0.5317 - accuracy: 0.7546
Epoch 1: val_accuracy improved from -inf to 0.93388, saving model to best_vgg19.h5
31/31 [==============================] - 35s 1s/step - loss: 0.5317 - accuracy: 0.7546 - val_loss: 0.2340 - val_accuracy: 0.9339
Epoch 2/50
31/31 [==============================] - ETA: 0s - loss: 0.3136 - accuracy: 0.8630
Epoch 2: val_accuracy did not improve from 0.93388
31/31 [==============================] - 31s 1s/step - loss: 0.3136 - accuracy: 0.8630 - val_loss: 0.2070 - val_accuracy: 0.9339
Epoch 3/50
31/31 [==============================] - ETA: 0s - loss: 0.2796 - accuracy: 0.8875
Epoch 3: val_accuracy did not improve from 0.93388
31/31 [==============================] - 31s 992ms/step - loss: 0.2796 - accuracy: 0.8875 - val_loss: 0.1740 - val_accuracy: 0.9339
Epoch 4/50
31/31 [==============================] - ETA: 0s - loss: 0.2585 - accuracy: 0.8957
Epoch 4: val_accuracy improved from 0.93388 to 0.

In [181]:
img_path ='/content/cataract-image-dataset/processed_images/test/cataract/image_305.png'
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg19.predict(np.array([x]))
if preds< 0.5:
    print('Cataract')
else:
    print('Normal')


1/1 [==============================] - 0s 18ms/step
Cataract


# Inception


In [182]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [183]:
inceptionV3 = InceptionV3(include_top=False ,input_shape = (224,224,3))

87910968/87910968 [==============================] - 1s 0us/step


In [184]:
for layer in inceptionV3.layers:
  print(layer)

In [185]:
len(inceptionV3.layers)

311

In [186]:
for layer in inceptionV3.layers:
  layer.trainable = False

In [187]:
x = Flatten()(inceptionV3.output)

In [188]:
output = Dense(1,activation = 'sigmoid')(x)

In [189]:
inceptionV3 =Model(inceptionV3.input,output)

In [190]:
inceptionV3.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_7[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0]

In [199]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'best_inception.h5',  # file name to save the best model
    monitor='val_accuracy',  # metric to monitor
    save_best_only=True,  # save only the best model
    mode='max',  # mode can be 'max' for accuracy, 'min' for loss, etc.
    verbose=1  # optional, set to 1 to see progress during training
)
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
opt = Adam(lr=0.0001)

# Assuming you have defined your VGG16 model as vgg16

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Compile the model (you may have already done this)
inceptionV3.compile(loss = 'binary_crossentropy',optimizer = opt,metrics =['accuracy'])

# Train the model with early stopping callback
inceptionV3.fit(training_set, validation_data=test_set, epochs=50, steps_per_epoch=len(training_set), validation_steps=len(test_set), callbacks=[checkpoint,early_stopping])

Epoch 1/50
31/31 [==============================] - ETA: 0s - loss: 0.3844 - accuracy: 0.9315
Epoch 1: val_accuracy improved from -inf to 0.93388, saving model to best_inception.h5
31/31 [==============================] - 39s 1s/step - loss: 0.3844 - accuracy: 0.9315 - val_loss: 0.5375 - val_accuracy: 0.9339
Epoch 2/50
31/31 [==============================] - ETA: 0s - loss: 0.1489 - accuracy: 0.9632
Epoch 2: val_accuracy improved from 0.93388 to 0.95041, saving model to best_inception.h5
31/31 [==============================] - 34s 1s/step - loss: 0.1489 - accuracy: 0.9632 - val_loss: 0.3126 - val_accuracy: 0.9504
Epoch 3/50
31/31 [==============================] - ETA: 0s - loss: 0.1142 - accuracy: 0.9714
Epoch 3: val_accuracy did not improve from 0.95041
31/31 [==============================] - 30s 975ms/step - loss: 0.1142 - accuracy: 0.9714 - val_loss: 0.3445 - val_accuracy: 0.9421
Epoch 4/50
31/31 [==============================] - ETA: 0s - loss: 0.1277 - accuracy: 0.9540
Epoch 

In [196]:
img_path ='/content/cataract-image-dataset/processed_images/test/cataract/image_300.png'

import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = inceptionV3.predict(np.array([x]))
preds
if preds< 0.5:
    print('Cataract')
else:
    print('Normal')

1/1 [==============================] - 1s 1s/step
Cataract


In [197]:
inceptionV3.save('inceptionV3_97.5%.h5')

# Xception

In [201]:
1from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [202]:
xception = Xception(include_top=False ,input_shape = (224,224,3))

In [203]:
for layer in xception.layers:
  print(layer)

In [204]:
len(xception.layers)

132

In [205]:
for layer in xception.layers:
  layer.trainable = False

In [206]:
x = Flatten()(xception.output)

In [207]:
output = Dense(1,activation = 'sigmoid')(x)

In [208]:
xception =Model(xception.input,output)

In [209]:
xception.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 111, 111, 32)         864       ['input_9[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 111, 111, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 111, 111, 32)         0         ['block1_conv1_bn[0][0]

In [215]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'best_exception.h5',  # file name to save the best model
    monitor='val_accuracy',  # metric to monitor
    save_best_only=True,  # save only the best model
    mode='max',  # mode can be 'max' for accuracy, 'min' for loss, etc.
    verbose=1  # optional, set to 1 to see progress during training
)
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
opt = Adam(lr=0.0001)


# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Compile the model (you may have already done this)
xception.compile(loss = 'binary_crossentropy',optimizer = opt,metrics =['accuracy'])

# Train the model with early stopping callback
xception.fit(training_set, validation_data=test_set, epochs=50, steps_per_epoch=len(training_set), validation_steps=len(test_set), callbacks=[checkpoint,early_stopping])

Epoch 1/50
31/31 [==============================] - ETA: 0s - loss: 0.3388 - accuracy: 0.9284
Epoch 1: val_accuracy improved from -inf to 0.95868, saving model to best_exception.h5
31/31 [==============================] - 37s 1s/step - loss: 0.3388 - accuracy: 0.9284 - val_loss: 0.2494 - val_accuracy: 0.9587
Epoch 2/50
31/31 [==============================] - ETA: 0s - loss: 0.4402 - accuracy: 0.9172
Epoch 2: val_accuracy did not improve from 0.95868
31/31 [==============================] - 32s 1s/step - loss: 0.4402 - accuracy: 0.9172 - val_loss: 0.4782 - val_accuracy: 0.9421
Epoch 3/50
31/31 [==============================] - ETA: 0s - loss: 0.3032 - accuracy: 0.9397
Epoch 3: val_accuracy did not improve from 0.95868
31/31 [==============================] - 32s 1s/step - loss: 0.3032 - accuracy: 0.9397 - val_loss: 0.4089 - val_accuracy: 0.9587
Epoch 4/50
31/31 [==============================] - ETA: 0s - loss: 0.2631 - accuracy: 0.9427
Epoch 4: val_accuracy did not improve from 0.958

In [216]:
img_path ='/content/cataract-image-dataset/processed_images/test/cataract/image_300.png'

import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = xception.predict(np.array([x]))
preds

if preds< 0.5:
    print('Cataract')
else:
    print('Normal')

1/1 [==============================] - 1s 676ms/step
Normal


# Resnet50

In [217]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [218]:
resnet50 = ResNet50(include_top = False,input_shape = (224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [219]:
for layer in resnet50.layers:
  print(layer)

In [220]:
len(resnet50.layers)

175

In [221]:
for layer in resnet50.layers:
  layer.trainable = False

In [222]:
x = Flatten()(resnet50.output)

In [223]:
output = Dense(1,activation = 'sigmoid')(x)

In [224]:
resnet50 =Model(resnet50.input,output)

In [225]:
resnet50.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_10[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [226]:
checkpoint = ModelCheckpoint(
    'best_exception.h5',  # file name to save the best model
    monitor='val_accuracy',  # metric to monitor
    save_best_only=True,  # save only the best model
    mode='max',  # mode can be 'max' for accuracy, 'min' for loss, etc.
    verbose=1  # optional, set to 1 to see progress during training
)
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
opt = Adam(lr=0.0001)

# Assuming you have defined your VGG16 model as vgg16

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Compile the model (you may have already done this)
resnet50.compile(loss = 'binary_crossentropy',optimizer = opt,metrics =['accuracy'])

# Train the model with early stopping callback
history = resnet50.fit(training_set, validation_data=test_set, epochs=50, steps_per_epoch=len(training_set), validation_steps=len(test_set), callbacks=[checkpoint,early_stopping])

Epoch 1/50
31/31 [==============================] - ETA: 0s - loss: 1.7782 - accuracy: 0.5143
Epoch 1: val_accuracy improved from -inf to 0.52066, saving model to best_exception.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 39s 1s/step - loss: 1.7782 - accuracy: 0.5143 - val_loss: 0.6800 - val_accuracy: 0.5207
Epoch 2/50
31/31 [==============================] - ETA: 0s - loss: 0.6523 - accuracy: 0.6053
Epoch 2: val_accuracy improved from 0.52066 to 0.66942, saving model to best_exception.h5
31/31 [==============================] - 32s 1s/step - loss: 0.6523 - accuracy: 0.6053 - val_loss: 0.6004 - val_accuracy: 0.6694
Epoch 3/50
31/31 [==============================] - ETA: 0s - loss: 0.6597 - accuracy: 0.6207
Epoch 3: val_accuracy did not improve from 0.66942
31/31 [==============================] - 32s 1s/step - loss: 0.6597 - accuracy: 0.6207 - val_loss: 0.6549 - val_accuracy: 0.5785
Epoch 4/50
31/31 [==============================] - ETA: 0s - loss: 0.6719 - accuracy: 0.6135
Epoch 4: val_accuracy did not improve from 0.66942
31/31 [==============================] - 30s 959ms/step - loss: 0.6719 - accuracy: 0.6135 - val_loss: 0.9273 - val_accuracy: 0.5207
Epoch 5